In [0]:
storage_account = "csvstoragedemo01"
container = "raw-csv"
sas_token = "sp=r&st=2025-09-13T13:26:24Z&se=2025-09-13T21:41:24Z&spr=https&sv=2024-11-04&sr=c&sig=dZinMuZrBMPpAivg%2BO3w9whkokYy4zc%2BjAUXUOwgTOU%3D"  

# Configure Databricks to access blob
spark.conf.set(f"fs.azure.sas.{container}.{storage_account}.blob.core.windows.net", sas_token)

csv_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/SalesOrderItems.csv"

# Read CSV
df = spark.read.csv(csv_path, header=True, inferSchema=True)
df.show(5)

+------------+--------------+---------+------+--------+-----------+---------+---------+-------------+---------+--------+------------+------------+
|SALESORDERID|SALESORDERITEM|PRODUCTID|NOTEID|CURRENCY|GROSSAMOUNT|NETAMOUNT|TAXAMOUNT|ITEMATPSTATUS|OPITEMPOS|QUANTITY|QUANTITYUNIT|DELIVERYDATE|
+------------+--------------+---------+------+--------+-----------+---------+---------+-------------+---------+--------+------------+------------+
|   500000000|            10|  MB-1034|      |     USD|  $2,499.00|$2,186.63|  $312.38|            I|     NULL|       4|          EA|    20180311|
|   500000000|            20|  CB-1161|      |     USD|    $399.00|  $349.13|   $49.88|            I|     NULL|       9|          EA|    20180311|
|   500000001|            10|  HB-1175|      |     USD|    $899.00|  $786.63|  $112.38|            I|     NULL|       2|          EA|    20180228|
|   500000001|            20|  RC-1056|      |     USD|  $2,499.00|$2,186.63|  $312.38|            I|     NULL|       

In [0]:
# List all files and folders in the container
files = dbutils.fs.ls(csv_path)

# Display the files
for file in files:
    print(file.name)


SalesOrderItems.csv


In [0]:
from pyspark.sql.functions import col, regexp_replace, to_date, date_format

df_clean = (
    df.select(
        col("SALESORDERID"),
        col("SALESORDERITEM").cast("int").alias("SALESORDERITEM"),
        col("PRODUCTID"),
        col("NOTEID").cast("int").alias("NOTEID"),
        col("CURRENCY"),
        regexp_replace(regexp_replace(col("GROSSAMOUNT"), "[$]", ""), ",", "").cast("double").alias("GROSSAMOUNT"),
        regexp_replace(regexp_replace(col("NETAMOUNT"), "[$]", ""), ",", "").cast("double").alias("NETAMOUNT"),
        regexp_replace(col("TAXAMOUNT"), "[$]", "").cast("double").alias("TAXAMOUNT"),
        col("ITEMATPSTATUS"),
        col("OPITEMPOS").cast("int").alias("OPITEMPOS"),
        col("QUANTITY").cast("int").alias("QUANTITY"),
        col("QUANTITYUNIT"),
        date_format(to_date(col("DELIVERYDATE").cast("string"), "yyyyMMdd"), "yyyy-MM-dd").alias("DELIVERYDATE")
        
)
)

df_clean.show(5)

+------------+--------------+---------+------+--------+-----------+---------+---------+-------------+---------+--------+------------+------------+
|SALESORDERID|SALESORDERITEM|PRODUCTID|NOTEID|CURRENCY|GROSSAMOUNT|NETAMOUNT|TAXAMOUNT|ITEMATPSTATUS|OPITEMPOS|QUANTITY|QUANTITYUNIT|DELIVERYDATE|
+------------+--------------+---------+------+--------+-----------+---------+---------+-------------+---------+--------+------------+------------+
|   500000000|            10|  MB-1034|  NULL|     USD|     2499.0|  2186.63|   312.38|            I|     NULL|       4|          EA|  2018-03-11|
|   500000000|            20|  CB-1161|  NULL|     USD|      399.0|   349.13|    49.88|            I|     NULL|       9|          EA|  2018-03-11|
|   500000001|            10|  HB-1175|  NULL|     USD|      899.0|   786.63|   112.38|            I|     NULL|       2|          EA|  2018-02-28|
|   500000001|            20|  RC-1056|  NULL|     USD|     2499.0|  2186.63|   312.38|            I|     NULL|       

In [0]:
container_name = 'raw-csv'
storage_account_name = 'csvstoragedemo01'

spark.conf.set("fs.azure.account.key.{storage_account}.blob.core.windows.net", "sp=rl&st=2025-09-05T08:21:14Z&se=2025-09-12T16:36:14Z&spr=https&sv=2024-11-04&sr=c&sig=8WdP5niQzFyAhIf2dtLki5tB%2FrP%2BCDqSbUkJv7SGols%3D")
csv_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/SalesOrders.csv"

# Read CSV
df = spark.read.csv(csv_path, header=True, inferSchema=True)
df.show(5)

+------------+---------+---------+---------+---------+-----------+----------------+------+---------+--------+--------+-----------+----------+----------+---------------+-------------+--------------+
|SALESORDERID|CREATEDBY|CREATEDAT|CHANGEDBY|CHANGEDAT|FISCVARIANT|FISCALYEARPERIOD|NOTEID|PARTNERID|SALESORG|CURRENCY|GROSSAMOUNT| NETAMOUNT| TAXAMOUNT|LIFECYCLESTATUS|BILLINGSTATUS|DELIVERYSTATUS|
+------------+---------+---------+---------+---------+-----------+----------------+------+---------+--------+--------+-----------+----------+----------+---------------+-------------+--------------+
|   500000000|        4| 20180111|        4| 20180116|         K4|         2018001|  NULL|100000022|     APJ|     USD| $13,587.00|$11,888.63| $1,698.38|              C|            C|             C|
|   500000001|        2| 20180112|        2| 20180115|         K4|         2018001|  NULL|100000026|    EMEA|     USD| $12,622.00|$11,044.25| $1,577.75|              C|            C|             C|
|   500000

In [0]:
from pyspark.sql.functions import col, regexp_replace, to_date, date_format

df_clean = (
    df.select(
        col("SALESORDERID").cast("string").alias("SALESORDERID"),
        col("CREATEDBY"),
        date_format(to_date(col("CREATEDAT").cast("string"), "yyyyMMdd"), "yyyy-MM-dd").alias("CREATEDATE"),
        col("CHANGEDBY"),
        date_format(to_date(col("CHANGEDAT").cast("string"), "yyyyMMdd"), "yyyy-MM-dd").alias("CHANGEDATE"),
        col("FISCVARIANT"),
        date_format(to_date(col("FISCALYEARPERIOD").cast("string"), "yyyyMMdd"), "yyyy-MM-dd").alias("FISCALYEARPERIOD"),
        col("NOTEID"),
        col("PARTNERID").cast("string").alias("PARTNERID"),
        col("SALESORG"),
        col("CURRENCY"),
        regexp_replace(regexp_replace(col("GROSSAMOUNT"), "[$]", ""), ",", "").cast("double").alias("GROSSAMOUNT"),
        regexp_replace(regexp_replace(col("NETAMOUNT"), "[$]", ""), ",", "").cast("double").alias("NETAMOUNT"),
        regexp_replace(col("TAXAMOUNT"), "[$]", "").cast("double").alias("TAXAMOUNT"),
        col("BILLINGSTATUS"),
        col("LIFECYCLESTATUS"),
        col("DELIVERYSTATUS")
        
        
)
)

df_clean.show(5)

+------------+---------+----------+---------+----------+-----------+----------------+------+---------+--------+--------+-----------+---------+---------+-------------+---------------+--------------+
|SALESORDERID|CREATEDBY|CREATEDATE|CHANGEDBY|CHANGEDATE|FISCVARIANT|FISCALYEARPERIOD|NOTEID|PARTNERID|SALESORG|CURRENCY|GROSSAMOUNT|NETAMOUNT|TAXAMOUNT|BILLINGSTATUS|LIFECYCLESTATUS|DELIVERYSTATUS|
+------------+---------+----------+---------+----------+-----------+----------------+------+---------+--------+--------+-----------+---------+---------+-------------+---------------+--------------+
|   500000000|        4|2018-01-11|        4|2018-01-16|         K4|            NULL|  NULL|100000022|     APJ|     USD|    13587.0| 11888.63|     NULL|            C|              C|             C|
|   500000001|        2|2018-01-12|        2|2018-01-15|         K4|            NULL|  NULL|100000026|    EMEA|     USD|    12622.0| 11044.25|     NULL|            C|              C|             C|
|   500000

In [0]:
container_name = 'raw-csv'
storage_account_name = 'csvstoragedemo01'

spark.conf.set("fs.azure.account.key.{storage_account}.blob.core.windows.net", "sp=rl&st=2025-09-05T08:21:14Z&se=2025-09-12T16:36:14Z&spr=https&sv=2024-11-04&sr=c&sig=8WdP5niQzFyAhIf2dtLki5tB%2FrP%2BCDqSbUkJv7SGols%3D")
csv_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/Products.csv"

# Read CSV
df = spark.read.csv(csv_path, header=True, inferSchema=True)
df.show(5)

+---------+--------+--------------+---------+---------+---------+---------+------------------+-------------+------------+-------------+----------+--------+-------+-----+-----+------+-------------+-------------+
|PRODUCTID|TYPECODE|PRODCATEGORYID|CREATEDBY|CREATEDAT|CHANGEDBY|CHANGEDAT|SUPPLIER_PARTNERID|TAXTARIFFCODE|QUANTITYUNIT|WEIGHTMEASURE|WEIGHTUNIT|CURRENCY|  PRICE|WIDTH|DEPTH|HEIGHT|DIMENSIONUNIT|PRODUCTPICURL|
+---------+--------+--------------+---------+---------+---------+---------+------------------+-------------+------------+-------------+----------+--------+-------+-----+-----+------+-------------+-------------+
|  RO-1001|      PR|            RO|        9| 20181003|        9| 20181003|         100000000|            1|          EA|          7.7|        KG|     USD|$525.00| NULL| NULL|  NULL|         NULL|         NULL|
|  RO-1002|      PR|            RO|        9| 20181003|        9| 20181003|         100000001|            1|          EA|          8.0|        KG|     USD|$

In [0]:
from pyspark.sql.functions import col, regexp_replace, to_date, date_format

df_clean = (
    df.select(
        col("PRODUCTID"),
        col("TYPECODE"),
        col("PRODCATEGORYID"),
        col("CREATEDBY"),
        date_format(to_date(col("CREATEDAT").cast("string"), "yyyyMMdd"), "yyyy-MM-dd").alias("CREATEDATE"),
        col("CHANGEDBY"),
        date_format(to_date(col("CHANGEDAT").cast("string"), "yyyyMMdd"), "yyyy-MM-dd").alias("CHANGEDATE"),
        col("SUPPLIER_PARTNERID").cast("int").alias("SUPPLIER_PARTNERID"),
        col("TAXTARIFFCODE"),
        col("QUANTITYUNIT"),
        col("WEIGHTMEASURE"),
        col("WEIGHTUNIT"),
        col("CURRENCY"),
        regexp_replace(regexp_replace(col("PRICE"), "[$]", ""), ",", "").cast("double").alias("PRICE"),
        col("WIDTH").cast("double").alias("WIDTH"),
        col("DEPTH").cast("double").alias("DEPTH"),
        col("HEIGHT").cast("double").alias("HEIGHT"),
        col("DIMENSIONUNIT"),
        col("PRODUCTPICURL")        
        
)
)

df_clean.show(5)

+---------+--------+--------------+---------+----------+---------+----------+------------------+-------------+------------+-------------+----------+--------+-----+-----+-----+------+-------------+-------------+
|PRODUCTID|TYPECODE|PRODCATEGORYID|CREATEDBY|CREATEDATE|CHANGEDBY|CHANGEDATE|SUPPLIER_PARTNERID|TAXTARIFFCODE|QUANTITYUNIT|WEIGHTMEASURE|WEIGHTUNIT|CURRENCY|PRICE|WIDTH|DEPTH|HEIGHT|DIMENSIONUNIT|PRODUCTPICURL|
+---------+--------+--------------+---------+----------+---------+----------+------------------+-------------+------------+-------------+----------+--------+-----+-----+-----+------+-------------+-------------+
|  RO-1001|      PR|            RO|        9|2018-10-03|        9|2018-10-03|         100000000|            1|          EA|          7.7|        KG|     USD|525.0| NULL| NULL|  NULL|         NULL|         NULL|
|  RO-1002|      PR|            RO|        9|2018-10-03|        9|2018-10-03|         100000001|            1|          EA|          8.0|        KG|     USD

In [0]:

spark.conf.set("fs.azure.account.key.{storage_account}.blob.core.windows.net", "sp=rl&st=2025-09-05T08:21:14Z&se=2025-09-12T16:36:14Z&spr=https&sv=2024-11-04&sr=c&sig=8WdP5niQzFyAhIf2dtLki5tB%2FrP%2BCDqSbUkJv7SGols%3D")
csv_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/ProductTexts.csv"

# Read CSV
df = spark.read.csv(csv_path, header=True, inferSchema=True)
df.show(5)

+---------+--------+----------------+------------+----------+
|PRODUCTID|LANGUAGE|     SHORT_DESCR|MEDIUM_DESCR|LONG_DESCR|
+---------+--------+----------------+------------+----------+
|  RO-1001|      EN|      Roady 1001|        NULL|      NULL|
|  RO-1002|      EN|      Roady 1002|        NULL|      NULL|
|  RO-1003|      EN|      Roady 1003|        NULL|      NULL|
|  BX-1011|      EN|BMX Vintage 1011|        NULL|      NULL|
|  BX-1012|      EN|   BMX Jump 1012|        NULL|      NULL|
+---------+--------+----------------+------------+----------+
only showing top 5 rows


In [0]:
csv_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/ProductCategories.csv"

# Read CSV
df = spark.read.csv(csv_path, header=True, inferSchema=True)
df.show(5)

+--------------+---------+---------+
|PRODCATEGORYID|CREATEDBY|CREATEDAT|
+--------------+---------+---------+
|            RO|       12| 20181003|
|            BX|        4| 20181003|
|            CC|        7| 20181003|
|            MB|       11| 20181003|
|            RC|        9| 20181003|
+--------------+---------+---------+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import col, to_date, date_format

df_clean = (
    df.select(
        col("PRODCATEGORYID"),
        col("CREATEDBY"),
        date_format(to_date(col("CREATEDAT").cast("string"), "yyyyMMdd"), "yyyy-MM-dd").alias("CREATEDATE"),       
        
)
)

df_clean.show(5)

+--------------+---------+----------+
|PRODCATEGORYID|CREATEDBY|CREATEDATE|
+--------------+---------+----------+
|            RO|       12|2018-10-03|
|            BX|        4|2018-10-03|
|            CC|        7|2018-10-03|
|            MB|       11|2018-10-03|
|            RC|        9|2018-10-03|
+--------------+---------+----------+
only showing top 5 rows


In [0]:
csv_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/ProductCategoryText.csv"

# Read CSV
df = spark.read.csv(csv_path, header=True, inferSchema=True)
df.show(5)

+--------------+--------+----------------+------------+----------+
|PRODCATEGORYID|LANGUAGE|     SHORT_DESCR|MEDIUM_DESCR|LONG_DESCR|
+--------------+--------+----------------+------------+----------+
|            RO|      EN|       Road Bike|        NULL|      NULL|
|            BX|      EN|             BMX|        NULL|      NULL|
|            CC|      EN|Cyclo-cross Bike|        NULL|      NULL|
|            MB|      EN|   Mountain Bike|        NULL|      NULL|
|            RC|      EN|     Racing Bike|        NULL|      NULL|
+--------------+--------+----------------+------------+----------+
only showing top 5 rows


In [0]:
csv_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/Employees.csv"

# Read CSV
df = spark.read.csv(csv_path, header=True, inferSchema=True)
df.show(5)

+----------+----------+-----------+---------+-------------+---+--------+--------------+--------------------+---------+----------+------------------+----------------+----+----+----+----+----+----+
|EMPLOYEEID|NAME_FIRST|NAME_MIDDLE|NAME_LAST|NAME_INITIALS|SEX|LANGUAGE|   PHONENUMBER|        EMAILADDRESS|LOGINNAME| ADDRESSID|VALIDITY_STARTDATE|VALIDITY_ENDDATE|_c13|_c14|_c15|_c16|_c17|_c18|
+----------+----------+-----------+---------+-------------+---+--------+--------------+--------------------+---------+----------+------------------+----------------+----+----+----+----+----+----+
|         1|   Derrick|          L|   Magill|         NULL|  M|       E|  630-374-0306|derrick.magill@it...| derrickm|1000000001|          20000101|        99991231|NULL|NULL|NULL|NULL|NULL|NULL|
|         2|   Philipp|          T|    Egger|         NULL|  M|       E|09603 61 24 64|philipp.egger@ite...| philippm|1000000002|          20000101|        99991231|NULL|NULL|NULL|NULL|NULL|NULL|
|         3|     Ell

In [0]:
from pyspark.sql.functions import col, regexp_replace, to_date, date_format, concat, lit

df_clean = (
    df.select(
        col("EMPLOYEEID"),
        col("NAME_FIRST"),
        col("NAME_MIDDLE"),
        col("NAME_LAST"),
        concat(col("NAME_FIRST"),lit(" "), col("NAME_MIDDLE"),lit(" "), col("NAME_LAST")).alias("EMPLOYEE_NAME"),
        col("NAME_INITIALS"),
        col("SEX"),
        col("LANGUAGE"),
        regexp_replace(regexp_replace(col("PHONENUMBER"),"[-]","")," ","").alias("PHONENUMBER"),
        col("EMAILADDRESS"),
        col("LOGINNAME"),
        col("ADDRESSID"),
        date_format(to_date(col("VALIDITY_STARTDATE").cast("string"), "yyyyMMdd"), "yyyy-MM-dd").alias("VALIDITY_STARTDATE"),
        date_format(to_date(col("VALIDITY_ENDDATE").cast("string"), "yyyyMMdd"), "yyyy-MM-dd").alias("VALIDITY_ENDDATE")
)
)

df_clean.drop(col("_c13"), col("_c14"), col("_c15"), col("_c16"), col("_c17"), col("_c18"))
df_clean.show(5)

+----------+----------+-----------+---------+-----------------+-------------+---+--------+-----------+--------------------+---------+----------+------------------+----------------+
|EMPLOYEEID|NAME_FIRST|NAME_MIDDLE|NAME_LAST|    EMPLOYEE_NAME|NAME_INITIALS|SEX|LANGUAGE|PHONENUMBER|        EMAILADDRESS|LOGINNAME| ADDRESSID|VALIDITY_STARTDATE|VALIDITY_ENDDATE|
+----------+----------+-----------+---------+-----------------+-------------+---+--------+-----------+--------------------+---------+----------+------------------+----------------+
|         1|   Derrick|          L|   Magill| Derrick L Magill|         NULL|  M|       E| 6303740306|derrick.magill@it...| derrickm|1000000001|        2000-01-01|      9999-12-31|
|         2|   Philipp|          T|    Egger|  Philipp T Egger|         NULL|  M|       E|09603612464|philipp.egger@ite...| philippm|1000000002|        2000-01-01|      9999-12-31|
|         3|     Ellis|          K|Robertson|Ellis K Robertson|         NULL|  M|       E|07086

In [0]:
csv_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/BusinessPartners.csv"

# Read CSV
df = spark.read.csv(csv_path, header=True, inferSchema=True)
df.show(5)

+---------+-----------+--------------------+-----------+---------+--------------------+----------+----------------+---------+---------+---------+---------+---------+--------+
|PARTNERID|PARTNERROLE|        EMAILADDRESS|PHONENUMBER|FAXNUMBER|          WEBADDRESS| ADDRESSID|     COMPANYNAME|LEGALFORM|CREATEDBY|CREATEDAT|CHANGEDBY|CHANGEDAT|CURRENCY|
+---------+-----------+--------------------+-----------+---------+--------------------+----------+----------------+---------+---------+---------+---------+---------+--------+
|100000000|          2|maria.brown@all4b...|  622734567|     NULL|http://www.all4bi...|1000000034|   All For Bikes|     Inc.|       10| 20181003|       10| 20181003|     USD|
|100000001|          2|bob.buyer@amazebi...|    3088530|     NULL|http://www.amazeb...|1000000035| Amaze Bikes Inc|     Inc.|       13| 20181003|       13| 20181003|     USD|
|100000002|          2|victor.sanchez@ar...| 3023352668|     NULL|http://www.arenas...|1000000036|Arena Sports Inc|     Inc.|

In [0]:
from pyspark.sql.functions import col, regexp_replace, to_date, date_format

df_clean = (
    df.select(
        col("PARTNERID").cast("string").alias("PARTNERID"),
        col("PARTNERROLE"),
        col("EMAILADDRESS"),
        col("PHONENUMBER"),
        col("FAXNUMBER"),
        col("WEBADDRESS"),
        col("ADDRESSID").cast("string").alias("ADDRESSID"),
        col("COMPANYNAME"),
        col("LEGALFORM"),
        col("CREATEDBY"),
        date_format(to_date(col("CREATEDAT"), "yyyyMMdd"), "yyyy-MM-dd").alias("CREATEDATE"),
        col("CHANGEDBY"),
        date_format(to_date(col("CHANGEDAT"), "yyyyMMdd"), "yyyy-MM-dd").alias("CHANGEDATE"),
        col("CURRENCY")        
        
)
)

df_clean.show(5)

+---------+-----------+--------------------+-----------+---------+--------------------+----------+----------------+---------+---------+----------+---------+----------+--------+
|PARTNERID|PARTNERROLE|        EMAILADDRESS|PHONENUMBER|FAXNUMBER|          WEBADDRESS| ADDRESSID|     COMPANYNAME|LEGALFORM|CREATEDBY|CREATEDATE|CHANGEDBY|CHANGEDATE|CURRENCY|
+---------+-----------+--------------------+-----------+---------+--------------------+----------+----------------+---------+---------+----------+---------+----------+--------+
|100000000|          2|maria.brown@all4b...|  622734567|     NULL|http://www.all4bi...|1000000034|   All For Bikes|     Inc.|       10|2018-10-03|       10|2018-10-03|     USD|
|100000001|          2|bob.buyer@amazebi...|    3088530|     NULL|http://www.amazeb...|1000000035| Amaze Bikes Inc|     Inc.|       13|2018-10-03|       13|2018-10-03|     USD|
|100000002|          2|victor.sanchez@ar...| 3023352668|     NULL|http://www.arenas...|1000000036|Arena Sports Inc|

In [0]:
csv_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/Addresses.csv"

# Read CSV
df = spark.read.csv(csv_path, header=True, inferSchema=True)
df.show(5)

+----------+----------+----------+----------------+--------+-------+------+-----------+------------------+----------------+---------+-----------+
| ADDRESSID|      CITY|POSTALCODE|          STREET|BUILDING|COUNTRY|REGION|ADDRESSTYPE|VALIDITY_STARTDATE|VALIDITY_ENDDATE| LATITUDE|  LONGITUDE|
+----------+----------+----------+----------------+--------+-------+------+-----------+------------------+----------------+---------+-----------+
|1000000034|West Nyack|     10994|   Settlers Lane|    5027|     US|  AMER|          2|          20000101|        99991231|41.100057| -73.973562|
|1000000035| Fair Oaks|     95628|Woodland Terrace|    4467|     US|  AMER|          2|          20000101|        99991231|38.638355|-121.286683|
|1000000036|      Dunn|     28334|   Layman Avenue|    2250|     US|  AMER|          2|          20000101|        99991231|35.312013|  -78.60995|
|1000000037|   Chicago|     60605|University Drive|    4697|     US|  AMER|          2|          20000101|        99991231|4

In [0]:
from pyspark.sql.functions import col, regexp_replace, to_date, date_format, concat, lit

df_clean = (
    df.select(
        col("ADDRESSID"),
        col("CITY"),
        col("POSTALCODE"),
        col("STREET"),
        col("BUILDING"),
        col("COUNTRY"),
        col("REGION"),
        concat(col("BUILDING"),lit(" "), col("STREET"),lit(" "), col("CITY"),lit(" "), col("POSTALCODE")).alias("ADDRESS"),
        col("ADDRESSTYPE"),
        date_format(to_date(col("VALIDITY_STARTDATE").cast("string"), "yyyyMMdd"), "yyyy-MM-dd").alias("VALIDITY_STARTDATE"),
        date_format(to_date(col("VALIDITY_ENDDATE").cast("string"), "yyyyMMdd"), "yyyy-MM-dd").alias("VALIDITY_ENDDATE"),
        col("LATITUDE"),
        col("LONGITUDE")
       
)
)


df_clean.show(5)

+----------+----------+----------+----------------+--------+-------+------+--------------------+-----------+------------------+----------------+---------+-----------+
| ADDRESSID|      CITY|POSTALCODE|          STREET|BUILDING|COUNTRY|REGION|             ADDRESS|ADDRESSTYPE|VALIDITY_STARTDATE|VALIDITY_ENDDATE| LATITUDE|  LONGITUDE|
+----------+----------+----------+----------------+--------+-------+------+--------------------+-----------+------------------+----------------+---------+-----------+
|1000000034|West Nyack|     10994|   Settlers Lane|    5027|     US|  AMER|5027 Settlers Lan...|          2|        2000-01-01|      9999-12-31|41.100057| -73.973562|
|1000000035| Fair Oaks|     95628|Woodland Terrace|    4467|     US|  AMER|4467 Woodland Ter...|          2|        2000-01-01|      9999-12-31|38.638355|-121.286683|
|1000000036|      Dunn|     28334|   Layman Avenue|    2250|     US|  AMER|2250 Layman Avenu...|          2|        2000-01-01|      9999-12-31|35.312013|  -78.60995

In [0]:
jdbc_url = "jdbc:sqlserver://ssmsserverbi.database.windows.net:1433;database=PowerbiDB;authentication=ActiveDirectoryPassword"

connection_props = {
  "user": "yourAADUser@yourdomain.com",
  "password": "<your AAD password>",
  "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}